# Декораторы

#### Функция - тоже объект 

In [1]:
def area(height, width):
    return height*width

def hypot(a,b):
    return (a**2+b**2)**0.5

print(area)
print(type(area))

<function area at 0x000001E8FB871828>
<class 'function'>


In [2]:
area2 = area
area2(1,2)

2

#### Можем перeдавать функцию, как аргумент

In [3]:
def call_two_times(func):
    func()
    func()

def call_n_times(func, n = 3):
    for _ in range(n):
        func()

def greet():
    print("Hello!")

call_n_times(greet,n=5)

Hello!
Hello!
Hello!
Hello!
Hello!


#### Можем создавать функции внутри других функций

In [4]:
def make_adder(n):
    def adder(a):
        return a+n
    return adder

adder_10 = make_adder(10)

print(adder_10)

print(adder_10(50))

<function make_adder.<locals>.adder at 0x000001E8FB871AF8>
60


#### Можем изменять функцию

In [5]:
def to_str(func):
    def new_func(arg):
        return f"Вывод функции: {func(arg)}"
    
    return new_func

def square(n):
    return n**2

print(square(32))

square = to_str(square)

print(square(32))

1024
Вывод функции: 1024


#### Декоратор - синтаксический сахар

In [6]:
@to_str
def square(n):
    return n**2

print(square(32))

Вывод функции: 1024


## 1. Редактирование аргументов и результатов

#### Округляем результат

In [8]:
def area(height, width):
    return height*width

def hypot(a,b):
    return (a**2+b**2)**0.5

print(area(1.234,1.72))
print(hypot(10,3))

2.12248
10.44030650891055


In [9]:
def area(height, width):
    return round(height*width, 4)

def hypot(a,b):
    return round((a**2+b**2)**0.5, 4)

print(area(1.234,1.72))
print(hypot(10,3))

2.1225
10.4403


In [10]:
def rounder(func):
    def wrap(arg1, arg2):
        result = func(arg1, arg2)
        return round(result, 5)
    
    return wrap

@rounder
def area(height, width):
    return height*width

@rounder
def hypot(a,b):
    return (a**2+b**2)**0.5

print(area(1.234,1.72))
print(hypot(10,3))

2.12248
10.44031


#### Проверка аргументов

In [11]:
def inverse(a):
    return 1/a

print(inverse(3))

0.3333333333333333


In [12]:
print(inverse(0))

ZeroDivisionError: division by zero

In [13]:
def zero_check(func):
    
    def wrapper(arg):
        if arg == 0:
            return "На ноль делить нельзя!!!"
        else:
            return func(arg)
    return wrapper

@zero_check
def inverse(a):
    return 1/a

print(inverse(3))
print(inverse(0))

0.3333333333333333
На ноль делить нельзя!!!


#### Комбинируем два декоратора

In [14]:
def rounder(func):
    def wrap(arg1):
        result = func(arg1)
        return round(result, 3)
    return wrap

def zero_check(func):
    def wrapper(arg):
        if arg == 0:
            return "На ноль делить нельзя!!!"
        else:
            return func(arg)
    return wrapper

@zero_check
@rounder
def inverse(a):
    return 1/a

print(inverse(3))
print(inverse(0))

0.333
На ноль делить нельзя!!!


## 2. Декоратор для функций от неопределенного количества аргументов 

In [15]:
def rounder(func):
    def wrap(*args, **kwargs):
        result = func(*args, **kwargs)
        return round(result,3)
    return wrap

@rounder
def inverse(a):
    return 1/a

@rounder
def hypot(a,b):
    return (a**2+b**2)**0.5

print(inverse(0.21))
print(hypot(2,5))

4.762
5.385


## 3. Параметризованный декоратор

In [16]:
def make_rounder(num_round = 3):
    def rounder(func):
        def wrap(*args, **kwargs):
            result = func(*args, **kwargs)
            return round(result, num_round)
        return wrap
    return rounder


@make_rounder(5)
def hypot(a,b):
    return (a**2+b**2)**0.5

print(hypot(2,5))

5.38516


## 4. Измеряем время работы функции

In [17]:
import time

def timer(func):
    def wrap(*args, **kwargs):
        t0 = time.time()
        result =  func(*args, **kwargs)
        t1 = time.time()
        print(f"Функция выполнялась {round(t1-t0,5)} секунд")
        return result
    

    return wrap

In [19]:
def fib(n):
    if n <= 2:
        return 1
    else:
        return fib(n-2)+fib(n-1)

@timer
def get_fib(n):
    return fib(n)

get_fib(31)

Функция выполнялась 0.79656 секунд


1346269

## 5. Измеряем количество вызовов функции

In [21]:
def counter(func):
    def wrap(*args, **kwargs):
        wrap.num_calls += 1
        result =  func(*args, **kwargs)
        return result
    wrap.num_calls = 0
    return wrap

@counter
def fib(n):
    if n <= 2:
        return 1
    else:
        return fib(n-2)+fib(n-1)

print(fib(7))
print(fib.num_calls)

13
25


In [22]:
@counter
def greet():
    print("Hello")

for i in range(10):
    greet()

Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello


In [23]:
greet.num_calls

10

## 6. Логгируем функцию

In [24]:
def logger(func):
    def wrap(*args, **kwargs):
        result = func(*args, **kwargs)
        log = f"args: {', '.join(map(str,args))} , result: {result}"
        wrap.logs.append(log)
        return result
    wrap.logs = []
    return wrap

@logger
def fib(n):
    if n <= 2:
        return 1
    else:
        return fib(n-2)+fib(n-1)
fib(10)

55

In [25]:
print(len(fib.logs))

109


## 7. Кэшируем функцию

In [32]:
def casher(func):
    def wrap(*args):
        if args in wrap.cashed:
            return wrap.cashed[args]
        else:
            result = func(*args)
            wrap.cashed[args] = result
            return result
    wrap.cashed = {}
    return wrap

@counter
@casher
def fib(n):
    if n <= 2:
        return 1
    else:
        return fib(n-2)+fib(n-1)

fib(100)

354224848179261915075

In [33]:
print(fib.num_calls)

197


In [34]:
def fib_2(n):
    a, b = 1, 1
    counter = 2
    while counter < n:
        a, b =b, a+b
        counter+=1
    return b

fib_2(10)

55

# Итераторы

### `iter(*итерируемый объект*)`

In [35]:
a = [1,2,3, 4, 5]

iterator = iter(a)

for i in iterator:
    print(i)

1
2
3
4
5


### `next(*итератор*)`

In [36]:
a = [1,2,3,10]

iterator = iter(a)

print(next(iterator))
print(next(iterator))
print(next(iterator))
print(next(iterator))

1
2
3
10


In [37]:
print(next(iterator))

StopIteration: 

In [38]:
a = [1,2,3,10]

iterator = iter(a)

print(next(iterator))
print(next(iterator))
print(next(iterator))
print(next(iterator))
print(next(iterator,"Конец!"))

1
2
3
10
Конец!


In [39]:
def generator():
    print("generator код до 1")
    yield 1
    print("generator код до 2")
    yield 2
    print("generator код до 3")
    yield 3
    print("generator код после 3")

generated = generator()

print(next(generated))
print(next(generated))
print(next(generated))
print(next(generated, "STOP"))

generator код до 1
1
generator код до 2
2
generator код до 3
3
generator код после 3
STOP


In [40]:
def get_fib(n):
    a, b = 1, 1
    yield a
    yield b
    while True:
        a, b = b, a+b
        if b > n:
            break
        yield b

for i in get_fib(4000):
    print(i)

1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584


In [41]:
def get_sum():
    print("Инициализация генератора")
    s = 0
    while True:
        s += yield s

g = (get_sum())
print("Создали генератор")
next(g)

Создали генератор
Инициализация генератора


0

In [42]:
print(g.send(1))
print(g.send(3))
print(g.send(10))
print(g.send(-2))

1
4
14
12
